# Initial visualizations of your data

This notebook is meant to let you quickly look at your post-manually sorted data

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pickle
import os
import sys
sys.path.append('/Users/brad/GentnerLab/code/gentnerlab/ephys-analysis/')
sys.path.append('/Users/brad/GentnerLab/code/NeuralTDA/src')
from ephys import core, events, rasters
import topology
import glob
%pylab inline

/Users/brad/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


Populating the interactive namespace from numpy and matplotlib


## Load the data

Set 'block_path' to the path of the folder containing your post-manually sorted data

In [ ]:
block_path = '/Users/brad/GentnerLab/data/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'
spikes     = core.load_spikes(block_path)
clusters   = core.load_clusters(block_path)
trials     = events.load_trials(block_path)
fs         = core.load_fs(block_path)
rec        = 0 

In [ ]:
len(clusters[clusters['quality']=='Good'])

In [ ]:
persistence_files = glob.glob(os.path.join(block_path, 'topology/50ms_avg_p/*.pkl'))
persistence_files
os.path.split(persistence_files[0])

## Plot rasters of each trial from each Good unit for all stims



In [ ]:
figsize = (22, 20)
fig, axs = plt.subplots(4, 4, figsize=figsize)
for stimind in range(16):
    tf = persistence_files[stimind]
    tfilepath, tfilename = os.path.split(tf)
    stimname = tfilename.split('_')[5]
    pdata = pickle.load(open(tf, 'r'))

    betti=1
    


    ax = axs.flatten()[stimind]
    for trial in range(20):
        trialdata = pdata['{}'.format(str(trial))]
        betti1 = np.zeros([len(trialdata), 2])
        for ind, filt in enumerate(trialdata):
            betti1[ind, 0] = filt[0]*0.05
            betti1[ind, 1] = filt[1][1]
        betti0 = np.zeros([len(trialdata), 2])
        for ind, filt in enumerate(trialdata):
            betti0[ind, 0] = filt[0]*0.05
            betti0[ind, 1] = filt[1][0]

        ax.set_xlim(0, 6.5)
        ax.set_ylim(0, 25)
        pltcolor='b'
        if stimname in ['stimA', 'stimB', 'stimC', 'stimD']:
            pltcolor = 'r'
        ax.plot(betti1[:, 0], betti1[:,1], pltcolor, lw=2)
        ax.set_title('Stimulus: {}'.format(stimname))
        ax.set_xlabel('Time (seconds)')
        ax.set_ylabel('First Betti Number')
plt.savefig('betti0_timedep_50ms.png')

In [ ]:
pdata

## See a cell / stim combo that looked interesting?  Plot it here!

In [ ]:
for stim in range(16):
    tf = persistence_files[stim]
    pdata = pickle.load(open(tf, 'r'))
    bettipts0 = []
    bettipts1 = []
    for trial in range(20):
        trialdata = pdata['{}'.format(str(trial))]
        betti1 = np.zeros([len(trialdata), 2])
        for ind, filt in enumerate(trialdata):
            betti1[ind, 0] = filt[0]*0.05
            betti1[ind, 1] = filt[1][1]
        betti0 = np.zeros([len(trialdata), 2])
        for ind, filt in enumerate(trialdata):
            betti0[ind, 0] = filt[0]*0.05
            betti0[ind, 1] = filt[1][0]
        bettipt = [betti0[-1, 1], betti1[-1, 1]]
        bettipts0.append(betti0[-1, 1])
        bettipts1.append(betti1[-1, 1])
    plt.figure()
    plt.scatter(bettipts0, bettipts1)

## Plot Average, Gaussian Smoothed PSTHs for each Unit

In [27]:
import test
import pandas as pd
reload(test)
times = np.array(range(0, 1000, 1))
testspikes = test.generate_ring_dataset(10, times, 0.02)
testtrials = pd.DataFrame({'time_samples': 0, 'stimulus_end': 1000, 'stimulus': 'test'}, index=[0])

(10, 1000)


[1,
 5,
 9,
 13,
 17,
 21,
 25,
 29,
 33,
 37,
 41,
 45,
 49,
 53,
 57,
 61,
 65,
 69,
 73,
 77,
 81,
 85,
 89,
 93,
 97]

## Plot rasters of each MUA unit

In [26]:
reload(rasters)
good_cluster_ids = range(10)

# We have 16 stims, so let's go with a 4x4 subplot layout
subplot_xy = [1,2]
figsize = (22,20)
period = [0,0]
rec=0
fs = 1000.0

for clusterID in good_cluster_ids:
    fig = rasters.plot_unit_raster(testspikes, testtrials, clusterID, period, rec, fs, subplot_xy, figsize)
    #raster_fname = os.path.join(raster_dir, 'unit{}_raster.png'.format(str(clusterID)))
    fig.savefig('ring_test{}.png'.format(str(clusterID)))
    plt.close(fig)